# 🌟 ORB Feature Detection - Demostración Completa
Este notebook demuestra el uso del detector ORB incluyendo:
1. Visualización de puntos clave
2. Invarianza a la rotación
3. Comparación de velocidad con SIFT
4. Stitching panorámico básico
5. Comparación visual con otros detectores
6. Aplicación potencial en SLAM


In [1]:

import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

def show_img(img, title=''):
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()


In [ ]:

img = cv2.imread('edificio.jpg', cv2.IMREAD_GRAYSCALE)
img_rot = cv2.imread('edificio_rotado.jpg', cv2.IMREAD_GRAYSCALE)
img1 = cv2.imread('paisaje1.jpg', cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('paisaje2.jpg', cv2.IMREAD_GRAYSCALE)


In [ ]:

orb = cv2.ORB_create()
kp = orb.detect(img, None)
img_kp = cv2.drawKeypoints(img, kp, None, color=(0,255,0))
show_img(img_kp, 'Puntos clave detectados con ORB')


In [ ]:

kp1, des1 = orb.detectAndCompute(img, None)
kp2, des2 = orb.detectAndCompute(img_rot, None)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
matched_rot = cv2.drawMatches(img, kp1, img_rot, kp2, matches[:20], None, flags=2)
show_img(matched_rot, 'Invarianza a la rotación (ORB)')


In [ ]:

sift = cv2.SIFT_create()
start = time.time()
kp_orb, des_orb = orb.detectAndCompute(img, None)
orb_time = time.time() - start

start = time.time()
kp_sift, des_sift = sift.detectAndCompute(img, None)
sift_time = time.time() - start

print(f"Tiempo ORB: {orb_time:.4f}s")
print(f"Tiempo SIFT: {sift_time:.4f}s")


In [ ]:

kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
matches = bf.match(des1, des2)
src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)
H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
result = cv2.warpPerspective(img1, H, (img1.shape[1]+img2.shape[1], img1.shape[0]))
result[0:img2.shape[0], 0:img2.shape[1]] = img2
show_img(result, 'Panorama con ORB')


In [ ]:

sift_kp = cv2.drawKeypoints(img, sift.detect(img, None), None, color=(255,0,0))
show_img(sift_kp, 'Puntos clave detectados con SIFT (comparación)')


In [ ]:

# Simulamos 2 frames de un SLAM básico
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)
slam_sim = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None, flags=2)
show_img(slam_sim, 'Simulación de SLAM con ORB (frame to frame matching)')


## 🚶‍♂️ Simulación de Movimiento tipo SLAM con Múltiples Frames

In [ ]:

import cv2
import numpy as np
import matplotlib.pyplot as plt

orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

frames = []
for i in range(1, 6):
    frame = cv2.imread(f'frame_{i}.jpg', cv2.IMREAD_GRAYSCALE)
    frames.append(frame)

for i in range(len(frames) - 1):
    img1 = frames[i]
    img2 = frames[i + 1]

    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    matched = cv2.drawMatches(img1, kp1, img2, kp2, matches[:20], None, flags=2)
    plt.imshow(matched)
    plt.title(f'Matching Frame {i+1} -> Frame {i+2}')
    plt.axis('off')
    plt.show()


### 🎞️ Animación del proceso tipo SLAM

In [ ]:
from IPython.display import Image
Image(filename="SLAM_robot_simulacion_flecha_fija.gif")